In [1]:
!rm -rf InkubaLM-Challenge
!git clone https://github.com/melissafasol/InkubaLM-Challenge.git
%cd InkubaLM-Challenge


Cloning into 'InkubaLM-Challenge'...
remote: Enumerating objects: 456, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 456 (delta 29), reused 25 (delta 13), pack-reused 409 (from 1)
Receiving objects: 100% (456/456), 1.46 MiB | 8.97 MiB/s, done.
Resolving deltas: 100% (303/303), done.
/content/InkubaLM-Challenge


In [88]:
%cd InkubaLM-Challenge
!git pull


[Errno 2] No such file or directory: 'InkubaLM-Challenge'
/content/InkubaLM-Challenge
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 346 bytes | 173.00 KiB/s, done.
From https://github.com/melissafasol/InkubaLM-Challenge
   c7d2565..d7ddda6  main       -> origin/main
Updating c7d2565..d7ddda6
Fast-forward
 utils/multitask_v2.py | 5 +----
 1 file changed, 1 insertion(+), 4 deletions(-)


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install -U trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install datasets

In [89]:
import sys
sys.path.append("..")  # Add parent directory to the path

import os
from typing import List
from pathlib import Path
import numpy as np

# DO NOT EDIT
# create submission file
import pandas as pd
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
)
from utils import (
    eval,
    model_function,
    multitask,
    experiment_logger,
    supplement,
    multitask_v2
    )

import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, concatenate_datasets, Dataset, Value
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

In [7]:
from google.colab import drive
drive.mount('/content/drive')
output_path = "/content/drive/MyDrive/InkubaLM-Challenge/Output"
os.makedirs(output_path, exist_ok=True)

Mounted at /content/drive


In [41]:
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

from huggingface_hub import login

try:
    from google.colab import userdata

    # Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
    # vars as appropriate for your system.
    # userdata.get("HF_TOKEN") indicates that the name of the token in the Colab env is HF_TOKEN
    os.environ["hf_token_2"] = userdata.get("hf_token_2")
except:
    os.environ["hf_token_2"] = "----"

login(token=os.environ["hf_token_2"])

token = os.environ["hf_token_2"]
if token == "----":
    print("⚠️ Warning: No Hugging Face token found. Some models may not load.")
else:
    login(token=token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [9]:
hf_token_2 = '..' # paste your token here
os.environ["HF_TOKEN"] = hf_token_2

## **Load and Explore Datasets**

In [122]:
print("# Loading datasets")
se_dataset, mt_dataset,xn_dataset = multitask_v2.load_and_combine_datasets("Train")
test_dataset = multitask_v2.load_and_combine_datasets("Test")


# Loading datasets
All Columns: ['data_source', 'premise', 'task', 'ID', 'instruction', 'langs', 'inputs', 'targets']
All Columns: ['data_source', 'premise', 'task', 'ID', 'instruction', 'langs', 'inputs', 'targets']


In [95]:
test = pd.DataFrame(test_dataset)

In [96]:
test

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,"{'ID': 'ID_f3c74c7b_sentiment_test__hausa', 't...","{'ID': 'ID_aad19dbf_sentiment_test__hausa', 't...","{'ID': 'ID_f6de0381_sentiment_test__hausa', 't...","{'ID': 'ID_cbec84fe_sentiment_test__swahili', ...","{'ID': 'ID_885caf5c_sentiment_test__hausa', 't...","{'ID': 'ID_7b906b9a_sentiment_test__hausa', 't...","{'ID': 'ID_c1b27b2b_sentiment_test__hausa', 't...","{'ID': 'ID_a5e55896_sentiment_test__swahili', ...","{'ID': 'ID_4ba46a40_sentiment_test__swahili', ...","{'ID': 'ID_d8cf7774_sentiment_test__hausa', 't...",...,"{'ID': 'ID_e7cee9ee_sentiment_test__swahili', ...","{'ID': 'ID_a95c4eda_sentiment_test__hausa', 't...","{'ID': 'ID_b5581ce7_sentiment_test__swahili', ...","{'ID': 'ID_c3eb5891_sentiment_test__hausa', 't...","{'ID': 'ID_055ec0b2_sentiment_test__swahili', ...","{'ID': 'ID_a540db97_sentiment_test__hausa', 't...","{'ID': 'ID_4b6adc9c_sentiment_test__swahili', ...","{'ID': 'ID_0af16a72_sentiment_test__swahili', ...","{'ID': 'ID_e477521e_sentiment_test__hausa', 't...","{'ID': 'ID_47503dde_sentiment_test__swahili', ..."
1,"{'ID': 'ID_290c7da3_test__mt_eng-hau', 'task':...","{'ID': 'ID_fced026b_test__mt_eng-hau', 'task':...","{'ID': 'ID_764be3cd_test__mt_eng-hau', 'task':...","{'ID': 'ID_929c7333_test__mt_eng-swa', 'task':...","{'ID': 'ID_a1b67fef_test__mt_eng-hau', 'task':...","{'ID': 'ID_a4bf2d63_test__mt_eng-hau', 'task':...","{'ID': 'ID_7ef4058c_test__mt_eng-hau', 'task':...","{'ID': 'ID_5a3d1671_test__mt_eng-swa', 'task':...","{'ID': 'ID_19b92b9d_test__mt_eng-swa', 'task':...","{'ID': 'ID_a9355f25_test__mt_eng-hau', 'task':...",...,"{'ID': 'ID_c6496989_test__mt_eng-swa', 'task':...","{'ID': 'ID_399e8e48_test__mt_eng-hau', 'task':...","{'ID': 'ID_d054836f_test__mt_eng-swa', 'task':...","{'ID': 'ID_e41159bb_test__mt_eng-hau', 'task':...","{'ID': 'ID_863222b7_test__mt_eng-swa', 'task':...","{'ID': 'ID_d8db936a_test__mt_eng-hau', 'task':...","{'ID': 'ID_c5df2cca_test__mt_eng-swa', 'task':...","{'ID': 'ID_eda1ff7d_test__mt_eng-swa', 'task':...","{'ID': 'ID_4b46d9cc_test__mt_eng-hau', 'task':...","{'ID': 'ID_06780aab_test__mt_eng-swa', 'task':..."
2,"{'ID': 'ID_f373d29e_test__afrixnli_hau', 'lang...","{'ID': 'ID_31e0a793_test__afrixnli_hau', 'lang...","{'ID': 'ID_4c44b62a_test__afrixnli_hau', 'lang...","{'ID': 'ID_d3568f00_test__afrixnli_swa', 'lang...","{'ID': 'ID_c1c6ab23_test__afrixnli_hau', 'lang...","{'ID': 'ID_3a1e7f50_test__afrixnli_hau', 'lang...","{'ID': 'ID_d5871deb_test__afrixnli_hau', 'lang...","{'ID': 'ID_8d0fbaf3_test__afrixnli_swa', 'lang...","{'ID': 'ID_b6d787b0_test__afrixnli_swa', 'lang...","{'ID': 'ID_90fae5cd_test__afrixnli_hau', 'lang...",...,"{'ID': 'ID_3cff9f76_test__afrixnli_swa', 'lang...","{'ID': 'ID_7f7fb68b_test__afrixnli_hau', 'lang...","{'ID': 'ID_4530d49c_test__afrixnli_swa', 'lang...","{'ID': 'ID_ad838acf_test__afrixnli_hau', 'lang...","{'ID': 'ID_6ace2d00_test__afrixnli_swa', 'lang...","{'ID': 'ID_88cd08fe_test__afrixnli_hau', 'lang...","{'ID': 'ID_404bc9af_test__afrixnli_swa', 'lang...","{'ID': 'ID_a399ed32_test__afrixnli_swa', 'lang...","{'ID': 'ID_9f28a032_test__afrixnli_hau', 'lang...","{'ID': 'ID_31eb3181_test__afrixnli_swa', 'lang..."


In [92]:
se_df = pd.DataFrame(se_dataset)
mt_df = pd.DataFrame(mt_dataset)
xn_df = pd.DataFrame(xn_dataset)

In [110]:
xn_df['task'] = 'afrixnli'
mt_df['task'] = 'mt'

In [79]:
synonyms = {
    "hau": {
        "nace": ["na fada", "na bayyana"],
        "ba": ["babu", "baya"],
        "ya": ["shi", "mutumin"],
    },
    "swa": {
        "kanuni": ["sheria", "mwongozo"],
        "maduka": ["duka", "bohari"],
        "habari": ["taarifa", "maelezo"],
    }
}

import pandas as pd
import random

def synonym_replace(text, lang, syn_dict):
    words = text.split()
    new_words = words[:]
    for i, word in enumerate(words):
        if word in syn_dict.get(lang, {}):
            new_words[i] = random.choice(syn_dict[lang][word])
            break
    return ' '.join(new_words)

def random_swap(text):
    words = text.split()
    if len(words) < 2:
        return text
    i, j = random.sample(range(len(words)), 2)
    words[i], words[j] = words[j], words[i]
    return ' '.join(words)

def random_deletion(text, p=0.1):
    words = text.split()
    if len(words) == 1:
        return text
    new_words = [w for w in words if random.random() > p]
    return ' '.join(new_words) if new_words else random.choice(words)

def apply_augmentation(row, synonyms):
    text = row["inputs"]
    lang = row["langs"]
    methods = [synonym_replace, random_swap, random_deletion]
    method = random.choice(methods)
    new_text = method(text, lang, synonyms) if method == synonym_replace else method(text)
    if new_text.strip() == text.strip():
        return None  # avoid duplicates
    new_row = row.copy()
    new_row["inputs"] = new_text
    new_row["is_augmented"] = True
    return new_row


In [102]:
# Load your data
df = xn_df
df["is_augmented"] = False

# Apply augmentation
augmented_rows = []
for _, row in df.iterrows():
    aug = apply_augmentation(row, synonyms)
    if aug is not None:
        augmented_rows.append(aug)

augmented_df = pd.DataFrame(augmented_rows)

# Combine and save
xnli_aug_df = pd.concat([xn_df, augmented_df], ignore_index=True)


In [124]:
test_dataset

(Dataset({
     features: ['ID', 'task', 'langs', 'data_source', 'instruction', 'inputs', 'targets', 'premise'],
     num_rows: 300
 }),
 Dataset({
     features: ['ID', 'task', 'langs', 'data_source', 'instruction', 'inputs', 'targets', 'premise'],
     num_rows: 300
 }),
 Dataset({
     features: ['ID', 'langs', 'premise', 'inputs', 'instruction', 'targets', 'data_source', 'task'],
     num_rows: 300
 }))

In [105]:
synonyms = {
    "hausa": {
        "kyakkyawa": ["mai kyau", "nagari"],
        "rahama": ["jinƙai", "tausayawa"],
        "korau": ["mummuna", "banza"],
        "jama'a": ["mutane", "al'umma"],
    },
    "swahili": {
        "zuri": ["safi", "ya kupendeza"],
        "mbaya": ["haifai", "bovu"],
        "hakuna": ["hamna", "siyo"],
        "chanya": ["nzuri", "ya kufurahisha"],
    }
}
def apply_augmentation(row, synonym_dict):
    text = row["inputs"]
    lang = row["langs"].strip().lower()

    methods = [synonym_replace, random_swap, random_deletion]
    method = random.choice(methods)

    new_text = method(text, lang, synonym_dict) if method == synonym_replace else method(text)

    if new_text.strip() == text.strip():
        return None  # No real change

    new_row = row.copy()
    new_row["inputs"] = new_text
    new_row["is_augmented"] = True
    return new_row


In [106]:
import pandas as pd

# Load data
df = se_df
df["is_augmented"] = False

# Apply augmentation
augmented_rows = []
for _, row in df.iterrows():
    aug = apply_augmentation(row, synonyms)
    if aug is not None:
        augmented_rows.append(aug)

aug_df = pd.DataFrame(augmented_rows)
sent_aug_df = pd.concat([se_df, aug_df], ignore_index=True)


In [111]:
#sent_aug_df = sent_aug_df.drop(columns=["is_augmented"], errors="ignore")
#xnli_aug_df = xnli_aug_df.drop(columns=["is_augmented"], errors="ignore")
train_df_concat = pd.concat([xnli_aug_df, sent_aug_df, mt_df], axis=0)

In [112]:
train_df_concat = train_df_concat.drop(columns=["is_augmented"], errors="ignore")

In [113]:
train_df_concat

,ID,langs,premise,inputs,instruction,targets,data_source,task
0,ID_648d37ff_dev_afrixnli_hau,hau,"Kuma kawai sai naji Hakane, wannan din ne.","Bayan nace e, ya ƙare.","Is the following question True, False or Neither?",0,,afrixnli
1,ID_f96a39cb_dev_afrixnli_hau,hau,"Eh, Eh, na sani, ba zan ma damu sosai ba idan ...",Ba zan damu ba idan ɗaukan nauyin kuɗin da ake...,"Is the following question True, False or Neither?",0,,afrixnli
2,ID_4c5d953b_dev_afrixnli_swa,swa,"Alafu, hakuelewa kabisa.","Alas, hakuweza kuelewa wazi kutokana na kizuiz...","Is the following question True, False or Neither?",1,,afrixnli
3,ID_99a61e3d_dev_afrixnli_hau,hau,"Kuma kawai sai naji Hakane, wannan din ne.",Nace a'a amma yaci gaba da ja-in-ja.,"Is the following question True, False or Neither?",2,,afrixnli
4,ID_a38aa9d8_dev_afrixnli_swa,swa,Lakini siamini chombo chochote cha algorithmic...,Kanuni haziwezi kuamua jinsi ya kutengeneza mk...,"Is the following question True, False or Neither?",1,,afrixnli
...,...,...,...,...,...,...,...,...
595,ID_b8ca5d86_dev_mt_eng-swa,eng-swa,,they must have known what was coming after int...,please convert this english content into swahili.,hawa walijua kitakachoendelea baada ya sheria ...,wmt22,mt
596,ID_7c9b5dbf_dev_mt_eng-swa,eng-swa,,a truly large amount of games and activities g...,please convert this english content into swahili.,"kwa kweli, mengi ya michezo na shughuli inakar...",wmt22,mt
597,ID_8aa26dab_dev_mt_eng-swa,eng-swa,,there are small businesses which can be succes...,could you convert this english text to swahili?,kwa hivyo vyama vidogo huenda vikafanikiwa kat...,wmt22,mt
598,ID_46056ad4_dev_mt_eng-hau,eng-hau,,identical twins are an example of natural clones.,could you convert this english text to hausa?,dabbobi da suka haifar da labaran su ne misala...,wmt22,mt


Analyse target sequence lengths (without balancing)

In [13]:
#unbalanced_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))

print("\n# The problem: Tasks with shorter outputs (like sentiment) will be underrepresented")
print("in the loss function compared to tasks with longer outputs (like mt).")


# The problem: Tasks with shorter outputs (like sentiment) will be underrepresented
in the loss function compared to tasks with longer outputs (like mt).


In [114]:
print("\n# Applying target length balancing fix")
balanced_df = multitask.balance_target_lengths(train_df_concat)

print("\n# Analyzing target sequence lengths after balancing")
task_stats_after = multitask.analyze_task_lengths(balanced_df)



# Applying target length balancing fix

# Analyzing target sequence lengths after balancing
Target sequence length analysis by task:
Task: afrixnli
count    627.0
mean      12.0
std        0.0
min       12.0
25%       12.0
50%       12.0
75%       12.0
max       12.0
Name: targets, dtype: float64
--------------------------------------------------
Task: sentiment
count    649.0
mean      12.0
std        0.0
min       12.0
25%       12.0
50%       12.0
75%       12.0
max       12.0
Name: targets, dtype: float64
--------------------------------------------------
Task: mt
count    600.000000
mean      11.801667
std        8.347163
min        2.000000
25%        6.000000
50%       10.000000
75%       15.000000
max       62.000000
Name: targets, dtype: float64
--------------------------------------------------


In [115]:
print("\n# Applying target length balancing fix")
#balanced_df = balance_target_lengths(train_df)
balanced_df = multitask_v2.balance_target_lengths(train_df_concat)


# Applying target length balancing fix


In [116]:
balanced_dataset = Dataset.from_pandas(balanced_df.reset_index(drop=True))

In [45]:
model_name = "lelapa/InkubaLM-0.4B"
# turn off qlora: use_4bit=False
model, tokenizer, bnb_config = multitask_v2.setup_model_and_tokenizer(model_name, token=token,use_4bit=False)

config.json:   0%|          | 0.00/763 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/991k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.95M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [117]:
balanced_dataset

Dataset({
    features: ['ID', 'langs', 'premise', 'inputs', 'instruction', 'targets', 'data_source', 'task'],
    num_rows: 1876
})

In [118]:
balanced_trainer = multitask_v2.setup_trainer(
    model=model,
    dataset=balanced_dataset,
    tokenizer=tokenizer,
    output_dir="./sft_model/balanced_heur_aug"
)
balanced_trainer.train()

Applying formatting function to train dataset:   0%|          | 0/1876 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1876 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1876 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1876 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1876 [00:00<?, ? examples/s]

Step,Training Loss
10,1.226200
20,1.415800
30,1.214500
40,1.652200
50,1.345500
60,1.311500
70,1.697000
80,1.195300
90,1.338000
100,1.380700


TrainOutput(global_step=351, training_loss=0.8384678409649775, metrics={'train_runtime': 997.3634, 'train_samples_per_second': 5.643, 'train_steps_per_second': 0.352, 'total_flos': 2352865562910720.0, 'train_loss': 0.8384678409649775})

In [119]:
output_dir = '/content/drive/MyDrive/InkubaLM/outputs/'

In [120]:
# Save base model (optional, if not already public)
#model.save_pretrained(os.path.join(output_dir, "sft_model/balanced/base_model"))

# Save LoRA adapter
model.save_pretrained(os.path.join(output_dir, "sft_model/balanced/lora_adapter_v3"))
tokenizer.save_pretrained(os.path.join(output_dir, "sft_model/balanced/lora_adapter_v3"))


('/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/tokenizer_config.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/special_tokens_map.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/tokenizer.model',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/added_tokens.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v3/tokenizer.json')

In [128]:
from datasets import concatenate_datasets

# Combine the tuple into a list
test_dataset_list = list(test_dataset)

# Concatenate all the datasets into one
combined_test_dataset = concatenate_datasets(test_dataset_list)

In [129]:
combined_test_dataset

Dataset({
    features: ['ID', 'task', 'langs', 'data_source', 'instruction', 'inputs', 'targets', 'premise'],
    num_rows: 900
})

In [130]:
v2_path = os.path.join(output_dir, "sft_model/balanced_supp/checkpoint-351")
os.makedirs(v2_path, exist_ok=True)
#inference_model = AutoModelForCausalLM.from_pretrained(ckpt_path,device_map="auto")
inference_model = model.eval()
results_df = multitask_v2.apply_inference_to_test_data(inference_model, tokenizer, combined_test_dataset)


Generating Responses:   0%|          | 0/900 [00:00<?, ?it/s]

In [131]:
results_df[['ID', 'Response']].to_csv(os.path.join(v2_path,'submission_full_finetune_v3.csv'), index=False)
results_df[['ID', 'generated','Response']].head()

,ID,generated,Response
0,ID_f3c74c7b_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2
1,ID_aad19dbf_sentiment_test__hausa,Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa,0
2,ID_f6de0381_sentiment_test__hausa,Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki,1
3,ID_cbec84fe_sentiment_test__swahili,Chanya Chanya Chanya Chanya Chanya Chanya Chan...,0
4,ID_885caf5c_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2


In [132]:
submission_csv = results_df[['ID','Response']]

In [43]:
submission_csv.to_csv(os.path.join(v2_path,'submission_full_finetune_v3.csv'), index=False)

In [133]:
test_csv = pd.read_csv(os.path.join(v2_path,'submission_full_finetune_v3.csv'))

In [134]:
test_csv

,ID,Response
0,ID_f3c74c7b_sentiment_test__hausa,2
1,ID_aad19dbf_sentiment_test__hausa,0
2,ID_f6de0381_sentiment_test__hausa,1
3,ID_cbec84fe_sentiment_test__swahili,0
4,ID_885caf5c_sentiment_test__hausa,2
...,...,...
895,ID_88cd08fe_test__afrixnli_hau,0
896,ID_404bc9af_test__afrixnli_swa,0
897,ID_a399ed32_test__afrixnli_swa,0
898,ID_9f28a032_test__afrixnli_hau,0
